<a href="https://colab.research.google.com/github/surajitdas2030/ML-Based-Output-Prediction-of-RTL-Model/blob/main/ParityPrediction/Parity_22bit_Git1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# only for Google Colab compatibiity
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import models, layers, activations
import random
print(tf.__version__)
np.random.seed(0)
tf.random.set_seed(0)

2.11.0


In [3]:
%matplotlib notebook

In [4]:
num_examples = 100000
num_classes = 32

#construct input values ,data set stored in the file: input_output_32_bit.txt 
def input_values():
    multiple_values = [map(int, '{0:032b}'.format(i)) for i in range(2 ** 22)]
    f = open("input_output_32_bit.txt", "w")
    random.shuffle(multiple_values)
    final_values = []
    for value in multiple_values[:num_examples]:
        temp = []
        f.write("[")
        count = 0
        for number in value:
            temp.append([number])
            count^=number
            f.write(str(number)+",")
        f.write("],\t")
        f.write("["+str(count)+"]\t"+"\n")
        final_values.append(temp)
    f.close
    return final_values
    
#calculate output values
def output_values(inputs):
    final_values = []
    for value in inputs:
        # output_values = [0 for _ in range(num_classes)]
        count = 0
        for i in value:
            count ^= i[0]
        # if count < num_classes:
            # output_values[count] = 1
        final_values.append([count])
    return final_values


In [5]:
x_samples = input_values()
y_samples = output_values(x_samples)

In [6]:
len(y_samples)


100000

In [7]:
x = np.array(x_samples) # 32 inputs
y = np.array(y_samples) # 1 ootput

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

In [9]:
#ANN1
#model = models.Sequential(name='Parity_checker')
#model.add(layers.Input(shape=(32,), name='input'))
#model.add(layers.Dense(units=32,activation=activations.relu, name='hidden1'))
## model.add(layers.Dense(units=3, activation=activations.sigmoid, name='hidden2'))
#model.add(layers.Dense(units=1, activation=activations.sigmoid, name='output'))

#add the layers
#ANN2:0.99 accuracy
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, input_dim=32, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
##model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0
_________________________________________________________________


In [10]:
#model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

Comparing the data and writing into the file ( **input_output_predict.txt**)

Here the
*  **x_text** is the input sample data
*  **y_text** is the expected output data
*  **y_predict** is predicted output data


In [11]:
model.fit(x_train, y_train, epochs=200)
score = model.evaluate(x_test, y_test, verbose=2)

Epoch 1/200
2188/2188 [==============================] - 12s 3ms/step - loss: 0.2505 - accuracy: 0.4985
Epoch 2/200
2188/2188 [==============================] - 8s 3ms/step - loss: 0.2500 - accuracy: 0.5048
Epoch 3/200
2188/2188 [==============================] - 8s 3ms/step - loss: 0.2499 - accuracy: 0.5061
Epoch 4/200
2188/2188 [==============================] - 7s 3ms/step - loss: 0.2498 - accuracy: 0.5120
Epoch 5/200
2188/2188 [==============================] - 7s 3ms/step - loss: 0.2497 - accuracy: 0.5140
Epoch 6/200
2188/2188 [==============================] - 7s 3ms/step - loss: 0.2496 - accuracy: 0.5127
Epoch 7/200
2188/2188 [==============================] - 7s 3ms/step - loss: 0.2493 - accuracy: 0.5177
Epoch 8/200
2188/2188 [==============================] - 7s 3ms/step - loss: 0.2489 - accuracy: 0.5233
Epoch 9/200
2188/2188 [==============================] - 7s 3ms/step - loss: 0.2486 - accuracy: 0.5253
Epoch 10/200
2188/2188 [==============================] - 7s 3ms/step - 

In [12]:
#Comparing the data and writing into the file ( input_output_predict.txt)
y_predict = model.predict(x_test).round()

f = open("input_output_predict.txt", "w")
n = y_test.size
cnt = 0
for i in range(0,n):
    f.write("[")
    count = 0
    for x in x_test[i]:
        for j in x:
          f.write(str(j)+",")
    f.write("];\t")
    f.write("["+str(y_test[i])+"];\t")
    f.write("["+str(y_predict[i])+"];\t \n")
    if y_predict[i]==y_test[i]:
      cnt+=1
f.write("Number of matches: "+str(cnt)+" out of " + str(n) +"\n")
f.close()
print("Number of matches: "+str(cnt)+" out of " + str(n) )

938/938 [==============================] - 2s 2ms/step
Number of matches: 29951 out of 30000
